<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Lesson 10: Bayesian Statistics and Likelihood</h1>


<a name='section_10_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L10.0 Overview</h2>


<h3>Navigation</h3>

<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_10_1">L10.1 Definition of Convolution</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_10_1">L10.1 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_10_2">L10.2 Example of Convolutions with Different Functions</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_10_2">L10.2 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_10_3">L10.3 Prior and Posterior Probabilities and Bayes Theorem</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_10_3">L10.3 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_10_4">L10.4 Bayesian vs. Frequentist and Likelihood</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_10_4">L10.4 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_10_5">L10.5 Bayesian vs. Frequentist Fitting Example</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_10_5">L10.5 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_10_6">L10.6 Maximum Likelihood</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_10_6">L10.6 Exercises</a></td>
    </tr>
</table>



In [ ]:
#>>>RUN: L10.0-runcell00

#install lmfit if you have not done so
!pip install lmfit

In [ ]:
#>>>RUN: L10.0-runcell01

import numpy as np                #https://numpy.org/doc/stable/ 
import lmfit                      #https://lmfit.github.io/lmfit-py/ 
import matplotlib.pyplot as plt   #https://matplotlib.org/3.5.3/api/_as_gen/matplotlib.pyplot.html
from scipy import stats           #https://docs.scipy.org/doc/scipy/reference/stats.html

In [ ]:
#>>>RUN: L10.0-runcell02

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title


<a name='section_10_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L10.1 Definition of Convolution</h2>  

| [Top](#section_10_0) | [Previous Section](#section_10_0) | [Exercises](#exercises_10_1) | [Next Section](#section_10_2) |


In [ ]:
#>>>RUN: L10.1-runcell01

#First let's define a triangular distribution
def triangle(x,mean=5):
    Norm=mean*mean
    val=np.where(x <= mean,np.maximum(x,np.zeros(len(x))), np.maximum(2*mean-x,np.zeros(len(x))))
    return val/Norm

#Now define the gaussian
def gaussian(x,mean=0,sigma=1):
    return 1./(sigma * np.sqrt(2 * np.pi)) * np.exp( - (x - mean)**2 / (2 * sigma**2)) 


In [ ]:
#>>>RUN: L10.1-runcell02

#Now let's do a convolution by hand
def convolve(f1,f2,x,sigma=1,iMin=-10,iMax=10,iN=2000):
    step=(iMax-iMin)/iN
    pInt=0
    for i0 in range(iN):
            pX   = np.repeat(i0*step+iMin,len(x))
            pVal = f1(x-pX,sigma=sigma)*f2(pX)
            pInt += pVal*step
    return pInt

#You could consider how the choice of iMin, iMax, and iN affect the output

In [ ]:
#>>>RUN: L10.1-runcell03

#now let's plot
fig, ax = plt.subplots()
x_in=np.linspace(-10, 15, 100)
tri_out=triangle(x_in)
gaus_out=gaussian(x_in)
conv1_out=convolve(gaussian,triangle,x_in)
conv2_out=convolve(gaussian,triangle,x_in,sigma=5)

ax.plot(x_in,tri_out,label='triangle')
ax.plot(x_in,gaus_out,label='gaussian')
ax.plot(x_in,conv1_out,label='convolved')
ax.plot(x_in,conv2_out,label='convolved($\sigma=5$)')
ax.set(xlabel='x(t)', ylabel='y(t)',title='Convolutions')
ax.grid()
plt.legend()
plt.show()

In [ ]:
#>>>RUN: L10.1-runcell04

#now let's plot the convolved and original wide Gaussian
fig, ax = plt.subplots()
x_in=np.linspace(-10, 15, 100)
tri_out=triangle(x_in)
gaus_out=gaussian(x_in,5,5)
conv2_out=convolve(gaussian,triangle,x_in,sigma=5)

ax.plot(x_in,tri_out,label='triangle')
ax.plot(x_in,gaus_out,label='gaussian')
ax.plot(x_in,conv2_out,label='convolved($\sigma=5$)')
ax.set(xlabel='x(t)', ylabel='y(t)',title='Convolutions')
ax.grid()
plt.legend()
plt.show()

<a name='exercises_10_1'></a>     

| [Top](#section_10_0) | [Restart Section](#section_10_1) | [Next Section](#section_10_2) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-10.1.1</span>

What happens if you convolve a Gaussian with a straight line: $y=x$? Complete the code below to define convolutions for two different values of sigma, using the previously defined functions 'gaussian' and 'convolve'. Then plot in your notebook.

Afterwards, consider the following: What happens to the value of the convolved function at $x=0$? What happens if you increase the width of the Gaussian?

In [ ]:
#>>>EXERCISE: L10.1.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

#defining a linear function
def line(x):
    #YOUR CODE HERE
    return

#creating the convolution
x_in=np.linspace(-10, 15, 100) #MUST USE THIS INPUT ARRAY FOR ANSWER CHECKER
conv1_out = pass #YOUR CODE HERE, use a gaussian with default mean and sigma
conv2_out = pass #YOUR CODE HERE, use a gaussian with default mean and sigma=5


#creating plots and comparing to the original functions
fig, ax = plt.subplots()

line_out=line(x_in)
gaus1_out=gaussian(x_in)

ax.plot(x_in,line_out,label='line')
ax.plot(x_in,gaus1_out,label='gaussian')
ax.plot(x_in,conv1_out,label='convolved')
ax.plot(x_in,conv2_out,label='convolved($\sigma=5$)')
ax.set(xlabel='x(t)', ylabel='y(t)',title='Convolutions')
ax.grid()
plt.legend()
plt.show()

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-10.1.2</span>

What happens if you convolve a Gaussian with a Gaussian? Complete the code below to define convolutions for two different values of sigma, again using the previously defined functions 'gaussian' and 'convolve'. Then plot in your notebook.

As an extra, consider the following questions:

- What is the functional form of the convolution?
- What happens to the value of the convolved function at $x=0$?
- What happens if you change the width of one or both of the Gaussians?

In [ ]:
#>>>EXERCISE: L10.1.2
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

#creating the convolution
x_in=np.linspace(-10, 15, 100) #MUST USE THIS INPUT ARRAY FOR ANSWER CHECKER
conv1_out = pass #YOUR CODE HERE, use a gaussian with default mean and sigma
conv2_out = pass #YOUR CODE HERE, use a gaussian with default mean and sigma=5


#creating plots and comparing to the original functions
fig, ax = plt.subplots()

gaus1_out=gaussian(x_in)

ax.plot(x_in,gaus1_out,label='gaussian')
ax.plot(x_in,conv1_out,label='convolved')
ax.plot(x_in,conv2_out,label='convolved($\sigma=5$)')
ax.set(xlabel='x(t)', ylabel='y(t)',title='Convolutions')
ax.grid()
plt.legend()
plt.show()

<a name='section_10_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L10.2 Example of Convolutions with Different Functions</h2>  

| [Top](#section_10_0) | [Previous Section](#section_10_1) | [Exercises](#exercises_10_2) | [Next Section](#section_10_3) |


In [ ]:
#>>>RUN: L10.2-runcell01

def func_1(x):
    return np.sin(x)

#this plots functions with two gaussians of different width
def plot_convolutions_with_gaussian(func_in,x,func_name):
    func_out=func_in(x)
    gaus_out=gaussian(x)
    conv1_out=convolve(gaussian,func_in,x)
    conv2_out=convolve(gaussian,func_in,x,sigma=2)

    fig, ax = plt.subplots()
    ax.plot(x,func_out,label=func_name)
    ax.plot(x,gaus_out,label='gaussian')
    ax.plot(x,conv1_out,label='convolved')
    ax.plot(x,conv2_out,label='convolved($\sigma=2$)')
    ax.set(xlabel='x(t)', ylabel='y(t)',title='Convolutions')
    ax.grid()
    plt.legend()
    plt.show()
    
    
#now let's plot
x_in=np.linspace(-10, 15, 100)
plot_convolutions_with_gaussian(func_1,x_in,'sin(x)')

In [ ]:
#>>>RUN: L10.2-runcell02

def func_2(x):
    return np.sin(x)*x


#now let's plot
x_in=np.linspace(-10, 15, 100)
plot_convolutions_with_gaussian(func_2,x_in,'sin(x)*x')

<a name='exercises_10_2'></a>     

| [Top](#section_10_0) | [Restart Section](#section_10_2) | [Next Section](#section_10_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-10.2.1</span>

Often we use convolutions to "smear" our distributions analytically, without having to build a simulation. In the code below we sample points from a Gaussian. Take the original function below (a box distribution), add points sampled by a Gaussian, and show that the distribution matches the convolution.

Specifically, complete the array `smeared_x_in`, which adds `x_in` and `smeared`. Then complete the array `smeared_box_out`, which takes the average of the smeared values from `func_box` evaluated at the `smeared_x_in` points. Only complete tese arrays in the answer checker, then run the code in your notebook to generate plots.

To clearly see that the distributions match, try changing which of `smeared_box_out` and `conv_out` gets plotted first, or just comment out one and then the other. The two should be identical, which means that you only see the one which is plotted second.

In [ ]:
#>>>EXERCISE: L10.2.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

np.random.seed(10)

#define a box function
def func_box(x):
    return 0+0.1*np.where(x < -10,0,1) - 0.1*np.where(x < 10,0,1)

lNToys=1000
nbins=100
sigma=1
smeared=np.random.normal(0,sigma,(lNToys,nbins))
x_in=np.linspace(-20, 20, nbins)

#add the smeared signal to x_in, make 1000 example x distributions from above
smeared_x_in=#YOUR CODE HERE

#average over 1000 sampled distributions
smeared_box_out=#YOUR CODE HERE 


#PLOT
#-------------
#now let's plot
fig, ax = plt.subplots()
x_in=np.linspace(-20, 20, 100)

#various functions to add to the final plot
box_out=func(x_in)
gaus_out=gaussian(x_in)
conv_out=convolve(gaussian,func,x_in)

#make final plot
ax.plot(x_in,box_out,label='box(x)')
ax.plot(x_in,smeared_box_out,label='smeared box(x)')
ax.plot(x_in,gaus_out,label='gaussian')
ax.plot(x_in,conv_out,label='convolved')
ax.set(xlabel='x(t)', ylabel='y(t)',title='Convolutions')
ax.grid()
plt.legend()
plt.show()


<a name='section_10_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L10.3 Prior and Posterior Probabilities and Bayes Theorem</h2>  

| [Top](#section_10_0) | [Previous Section](#section_10_2) | [Exercises](#exercises_10_3) | [Next Section](#section_10_4) |


In [ ]:
#>>>RUN: L10.3-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L10/slides_L10_03.html', width=970, height=550)

In [ ]:
#>>>RUN: L10.3-runcell01

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

#Our measurement probability
def gaus(mu=0,sigma=1): 
    x = np.arange(-10, 10, 0.001)
    y = stats.norm.pdf(x,mu,sigma)
    return x,y

#a quick plot of what we expect the measurement to be
def plotgaus():
    x,y=gaus(0,1)
    fig, ax = plt.subplots(figsize=(9,6))
    plt.style.use('fast')
    ax.plot(x,y)
    ax.fill_between(x,y,0, alpha=0.1, color='b')
    ax.set_xlim([-6,6])
    ax.set_xlabel('x')
    ax.set_ylabel('p')
    plt.show()

plotgaus()

In [ ]:
#>>>RUN: L10.3-runcell02

def gaus(mu=0,sigma=1,meas=2): 
    x = np.arange(-10, 10, 0.001)
    xmeas = np.arange(meas, 10, 0.001)
    y = stats.norm.pdf(x,mu,sigma)
    ymeas = stats.norm.pdf(xmeas,mu,sigma)
    return x,y,xmeas,ymeas

def plotgaus():
    x,y,xmeas,ymeas=gaus(0,1,2)
    fig, ax = plt.subplots(figsize=(9,6))
    plt.style.use('fast')
    ax.plot(x,y)
    ax.fill_between(x,y,0, alpha=0.1, color='b')
    ax.fill_between(xmeas,ymeas,0, alpha=0.3, color='b')
    ax.set_xlim([-6,6])
    ax.set_xlabel('x')
    ax.set_ylabel('p')
    plt.show()
    
plotgaus()

In [ ]:
#>>>RUN: L10.3-runcell03

np.random.seed(32)

def gaus(mu=0,sigma=1): 
    x = np.arange(-10, 10, 0.001)
    y = stats.norm.pdf(x,mu,sigma)
    return x,y

def plotGausSample(iZ,iSample):
    plt.style.use('fast')
    fig, ax = plt.subplots(figsize=(9,6))
    #sample
    samples = np.random.normal(iZ,1,iSample)
    x,y=gaus(iZ,1)
    #
    xs,ys=gaus(0,1)
    ax.plot(xs,ys,label='prior')
    ax.plot(x,y,label='posterior')
    count, bins, ignored = plt.hist(samples, 30, density=True, label=str(iSample)+' samples')
    ax.fill_between(xs,ys,0, alpha=0.1)
    ax.set_xlim([-6,6])
    ax.set_xlabel('x')
    ax.set_ylabel('p')
    ax.legend()
    plt.show()

plotGausSample(2,10)
plotGausSample(2,1000)

In [ ]:
#>>>RUN: L10.3-runcell04

np.random.seed(32)

def plotGausSample(iZ,iSample):
    plt.style.use('fast')
    #sample
    samples = np.random.normal(iZ,1,iSample)
    x,y=gaus(iZ,1)
    #noral st
    xs,ys=gaus(0,1)
    ys*=100
    fig, ax = plt.subplots(figsize=(9,6))
    ax.plot(xs,ys,label='prior')
    count, bins, ignored = plt.hist(samples, 30, density=True, label=str(iSample)+' samples')
    ax.plot(x,y,label='posterior')
    ax.fill_between(xs,ys,0, alpha=0.1)
    ax.set_xlim([-6,6])
    ax.set_xlabel('x')
    ax.set_ylabel('p')
    ax.legend(loc=2)
    #plt.ylim(0,0.43) #note: these bounds are used in video
    plt.ylim(0,3.0) #note: these bounds more clearly shows what we are investigating
    plt.show()
    
plotGausSample(2,10)

plotGausSample(2,1000)


<a name='exercises_10_3'></a>     

| [Top](#section_10_0) | [Restart Section](#section_10_3) | [Next Section](#section_10_4) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-10.3.1</span>

If you roll a normal die and observe the outcome 6 three times in a row, are you surprised? What is the prior in this case? Enter the value for your prior as a fraction with precision 1e-4.

Hint: You could use `stats.binom.pmf` to compute this value.

In [ ]:
#>>>EXERCISE: L10.3.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-10.3.2</span>

Let's say your prior is a Gaussian with mean 0 and width 1, but your true, posterior distribution is a Gaussian with mean 0.5 and width 1. Generate 1000 events using this posterior distribution and find the most extreme, positive value (farthest from 0). What is the p-value of this extreme value (i.e., the probability of observing this extreme value or higher), given your prior? Enter your answer as a number with precision 1e-6.

Given that you have 1000 samples from this posterior distribution, how does this probability compare with what you would expect? In other words, is it likely to have occured once in 1000 samples?

In [ ]:
#>>>EXERCISE: L10.3.2
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

#use this random seed when calculating/reporting your answers
np.random.seed(101)

#1000 random samples from posterior distribution
sample=#YOUR CODE HERE

#most extreme value from samples (take the largest absolute value)
exvalue=#YOUR CODE HERE

#the probability of attaining that extreme value (or greater), given your prior
pvalue=#YOUR CODE HERE

#print results
print("extreme value",exvalue,"pvalue",pvalue)

<a name='section_10_4'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L10.4 Bayesian vs. Frequentist and Likelihood</h2>  

| [Top](#section_10_0) | [Previous Section](#section_10_3) | [Exercises](#exercises_10_4) | [Next Section](#section_10_5) |


In [ ]:
#>>>RUN: L10.4-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L10/slides_L10_04.html', width=970, height=550)

In [ ]:
#>>>RUN: L10.4-runcell01

#Our measurement probability
def gaus(mu=0,sigma=1): 
    x = np.arange(-10, 10, 0.001)
    y = stats.norm.pdf(x,mu,sigma)
    return x,y

def plotGausSampleLike(iZ,iSample):
    plt.style.use('fast')
    fig, ax = plt.subplots(figsize=(9,6))
    #Sample
    samples = np.random.normal(iZ,1,iSample)
    x,y=gaus(iZ,1)
    #prior
    xs,ys=gaus(0,1)
    #likelihood
    yratio=np.minimum(y/(ys*20.),1.)
    #plot
    ax.plot(xs,ys,label='prior')
    ax.plot(x,y,label='posterior')
    ax.plot(x,yratio,label='Likelihood/20')
    count, bins, ignored = plt.hist(samples, 30, density=True)
    ax.fill_between(xs,ys,0, alpha=0.1)
    ax.set_xlim([-6,6])
    ax.set_xlabel('x')
    ax.set_ylabel('p')
    ax.legend()
    plt.show()

plotGausSampleLike(2,1000)

In [ ]:
#>>>RUN: L10.4-runcell02

def plotGausSampleLikeNew(iZ,iSample,iSigMax):
    plt.style.use('fast')
    fig, ax = plt.subplots(figsize=(9,6))
    #ample our posterior
    samples = np.random.normal(iZ,1,iSample)
    x,y=gaus(iZ,1)
    #Sample our prior
    xs,ys=gaus(0,iSigMax) #######<<<<< This is our tweak
    #now compute the likelihood
    yratio=np.minimum(0.05*y/ys,20.)
    #plot this stuff
    ax.plot(xs,ys,label='prior')
    ax.plot(x,y,label='posterior')
    ax.plot(x,yratio,label='Likelihood/20')
    count, bins, ignored = plt.hist(samples, 30, density=True)
    ax.fill_between(xs,ys,0, alpha=0.1)
    ax.set_xlim([-6,6])
    ax.set_xlabel('x')
    ax.set_ylabel('p')
    ax.legend()
    plt.show()
    #now return our sampled normal distribution
    return samples
    
samples=plotGausSampleLikeNew(2,1000,2)

<a name='exercises_10_4'></a>     

| [Top](#section_10_0) | [Restart Section](#section_10_4) | [Next Section](#section_10_5) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-10.4.1</span>

The choice of prior can really change your outlook on fittings. Here you will compare the likelihood using two different priors.

First, plot the maximum likelihood from -5 to 5, using a posterior Gaussian distribution centered at 0.5 and a prior Gaussian distribution centered at 0. Next, use a prior that is a flat distribution, and compare the new likelihood ratio to your previous result. Which prior would be better to use, and why? Choose the best answer from the following:

A) Using a Gaussian prior yields a likelihood ratio that is also Gaussian.\
B) Using a Gaussian prior yields a likelihood ratio that is easier to interpret.\
C) Using a flat prior ensures that the likelihood ratio is a contant value.\
D) Using a flat prior ensures that the likelihood ratio does not skyrocket.

In [ ]:
#>>>EXERCISE: L10.4.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def flat(iX):
    return np.where(iX < 5, np.where(iX < -5, 0, 0.1 ), 0)

def plotGausSampleLikeNew2(iPost,iPrior):
    plt.style.use('fast')
    fig, ax = plt.subplots(figsize=(9,6))
    
    #Base Posterior a function
    x,y=gaus(iPost,1)
        
    #Gaussian prior
    xs,ys = ### Add code here    

    #Flat Prior
    yflat = ### Add code here    
    
    #now compute the likelihood ratio
    #use 20. as a maximum value
    ygaussratio = ### add code here
    yflatratio = ### add code here
    
    #plot this stuff
    ax.plot(x,y,label='posterior')
    
    #plot Gaussian prior and likelihood
    ax.plot(xs,ys,label='Gaussian prior')
    ax.fill_between(xs,ys,0, alpha=0.1)
    ax.plot(x,yratio/20.,label='Gaussian Likelihood/20')
    
    #plot flat prior and likelihood
    ax.plot(x,yflat,label='Flat prior')
    ax.plot(x,yflatratio/20.,label='Flat Likelihood/20')
    
    ax.set_xlim([-5,5])
    ax.set_xlabel('x')
    ax.set_ylabel('p')
    ax.legend()
    plt.show()
    #now return our sampled normal distribution
    return samples
    
samples=plotGausSampleLikeNew2(2,0)


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-10.4.2</span>

Now let's calculate the likelihood ratio for the maximum value obtained from simulated data, where the data are randomly drawn from a Gaussian distribution with mean=0.25 and sigma=1. We will use a Gaussian prior with mean=0 and sigma=1. 

First, complete the code, which should do the following: the `maxlikelihood` function should calculate the likelihood ratio between two Gaussian distributions (posterior/prior), evaluated at the value `val`, where `val` is the maximum value obtained from the randomly sampled array`isamples`. In the function `maxlike`, the array `samples` is drawn from a Gaussian distribution with mean=0.25 and sigma=1, and `plotGausSampleLikeNew` is used to obtain the data and make the plots.

After completing the code, compare the output of `maxlike` using 1e2 samples and 1e6 samples. What is the maximum likelihood value in each case? Would you expect it to increase or decrease with more samples? What does this mean about the distributions (do we have the right prior)?

Report your answer as a list of two numbers with precition 1e-2: `[max likelihood 1e2 samples, max likelihood 1e6 samples]`



In [ ]:
#>>>EXERCISE: L10.4.2
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

np.random.seed(3)

def maxlikelihood(isamples,mu1=0.25,sig1=1,mu2=0,sig2=1):
    val=np.max(isamples)#compute the highest sampled gaussian
    #now compute the likelihood of these two
    like=#YOUR CODE HERE
    return like

def maxlike(iN):
    samples=plotGausSampleLikeNew(0.25,iN,1)
    like=maxlikelihood(samples)
    print("Max likelihood:",iN," is ",like)

maxlike(100)
maxlike(1000000)

<a name='section_10_5'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L10.5 Bayesian vs. Frequentist Fitting Example</h2>  

| [Top](#section_10_0) | [Previous Section](#section_10_4) | [Exercises](#exercises_10_5) | [Next Section](#section_10_6) |


In [ ]:
#>>>RUN: L10.5-runcell01

#with Bayesian, we hypothesize a Gaussian and fit it
from lmfit.models import GaussianModel

np.random.seed(42)

#randomly sample 100 events from a Gaussian
lN=100
samples = np.random.normal(0,1,lN)
#make a histogram
count, bins, ignored = plt.hist(samples,30)
binscenters = np.array([0.5 * (bins[i] + bins[i+1]) for i in range(len(bins)-1)])
#poisson unc.
weight=1./np.sqrt(count)
weight[weight==float('+inf')] = 0
plt.show()

#Now we can consider two ways to interpret the data
def frequentist(iBins,iCount,weight): #fit a Gaussian floating all parameters
    model = GaussianModel()
    params = model.make_params(center=2, amplitude=1, sigma=1) 
    result = model.fit(iCount, params, x=iBins,weights=weight)
    result.plot()
    print(result.fit_report())
    

frequentist(binscenters,count,weight)


In [ ]:
#>>>RUN: L10.5-runcell02

np.random.seed(42)
    
def bayesianBad(iBins,iCount,weight):#fit a gaussian fix the mean and sgima
    model = GaussianModel()
    params = model.make_params(center=2, amplitude=1, sigma=1) 
    params['center'].vary=False
    params['sigma'].vary=False
    result = model.fit(iCount, params, x=iBins,weights=weight)
    result.plot()
    print(result.fit_report())

bayesianBad(binscenters,count,weight)

In [ ]:
#>>>RUN: L10.5-runcell03

np.random.seed(42)

import lmfit
#here is our modified function
def gauss(x, amp, mu, sigma,dmu):
    return amp * np.exp(-(x-mu+dmu)**2 / (2.*sigma**2))

#now we define our loss we want to minimize
def resid(params, x, ydata,weights):
    mu    = params['center'].value
    sigma = params['sigma'].value
    amp   = params['amplitude'].value
    dmu   = params['deltamu'].value
    lossshift=0
    if abs(dmu) > 3:
        lossshift=1e32
    y_model= gauss(x,amp,mu,sigma,dmu)
    residarr = (y_model - ydata)*weights
    #now append our constraint to the loss
    residarr = np.append(residarr,lossshift)
    return residarr
    
def bayesianGood(iBins,iCount,weights,initial=2):
    model = GaussianModel()
    params = model.make_params(center=initial, amplitude=1, sigma=1) 
    params['center'].vary=False
    params['sigma'].vary=False
    params.add("deltamu", value=0.0, min=-10, max=10) #Our new line of code
    result = lmfit.minimize(resid, params, args=(iBins, iCount,weights))
    lmfit.report_fit(result)
    #Now we plot it. 
    plt.errorbar(iBins, iCount,np.sqrt(iCount), lw=2,fmt=".k", capsize=0)
    plt.plot(binscenters,gauss(binscenters,result.params['amplitude'].value,result.params['center'].value,result.params['sigma'].value,result.params['deltamu'].value))
    plt.xlabel("x")
    plt.ylabel("p")
    plt.show()

bayesianGood(binscenters,count,weight)



<a name='exercises_10_5'></a>     

| [Top](#section_10_0) | [Restart Section](#section_10_5) | [Next Section](#section_10_6) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-10.5.1</span>

Run the above Bayesian good fit, centering the mean at an incorrect value of 1 and then the correct value 0 (this can be set using the `initial` parameter in the function).

What is the bias on mu (i.e., `deltamu`) in each case? Is it significant? Report your answer as a list of two numbers with precision 1e-2: `[deltamu1, deltamu0]`.


In [ ]:
#>>>EXERCISE: L10.5.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

np.random.seed(42)
#YOUR CODE HERE

<a name='section_10_6'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L10.6 Maximum Likelihood</h2>     

| [Top](#section_10_0) | [Previous Section](#section_10_5) | [Exercises](#exercises_10_6) |


<a name='exercises_10_6'></a>   

| [Top](#section_10_0) | [Restart Section](#section_10_6) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-10.6.1</span>

Which of the following statements best explains why we maximize the likelihood function in statistical inference?

A) To obtain estimates of unknown parameters based on observed data\
B) To minimize the sum of squared errors between the predicted and actual values\
C) To test the hypothesis that the data was generated by a particular model\
D) To calculate the mean and variance of the data
